In [ ]:
#FINAL PROJECT CODEBOOK

In [4]:
#PSEUDOCODE:

#1: Add in permit, census, & eviction data

#2: Clean eviction & permit data to only include pg evictions & "student-oriented" buildings 

#3: Join student-oriented permits & eviction data to census tracts

#4: Create map with census markers as base & permits & evictions as overlaid points

#5: Conduct analysis to determine spatial relationship/correlation

In [2]:
#1: DATA GATHERING

#1a: Import packages
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os


In [14]:
#1b: Read in data

#permits
epermits = pd.read_csv("EPermit.csv")
momentum = pd.read_csv("Momentum.csv")

#evictions
evictions = gpd.read_parquet('md_eviction_warrants.geoparquet')

#census
crs = 26918 # UTM18
poverty = gpd.read_file('pg_poverty_2023_5yr.geojson').to_crs(crs)
income = gpd.read_file('pg_income_2023_5yr.geojson').to_crs(crs)
population = gpd.read_file('pg_population_2023_5yr.geojson').to_crs(crs)

/var/folders/vh/xh16958j4mn34567ww18s3g00000gn/T/ipykernel_35864/1395333704.py:5: DtypeWarning: Columns (12,13,14,20,27,42) have mixed types. Specify dtype option on import or set low_memory=False.
  momentum = pd.read_csv("Momentum.csv")


In [5]:
evictions.columns.tolist()

['ID',
 'EventDate',
 'EventType',
 'EventComment',
 'County',
 'Location',
 'TenantAddress',
 'TenantCity',
 'TenantState',
 'TenantZipCode',
 'CaseType',
 'CaseNumber',
 'EvictedDate',
 'Source',
 'SourceDate',
 'Year',
 'EvictionYear',
 'unique_id',
 'input_address',
 'match_status',
 'match_type',
 'match_address',
 'match_lon_lat',
 'match_tiger_line_id',
 'match_tiger_line_side',
 'lon',
 'lat',
 'geometry']

In [7]:
#epermits['PROPOSED_USE'].unique()

In [9]:
#1c: clean eviction data
eviction_tidy = evictions[evictions['County'].isin(['Prince George\'s']) & evictions['EventType'].isin(['Warrant of Restitution - Return of Service - Evicted', 'warrant_evicted'])]
eviction_tidy.head()


,ID,EventDate,EventType,EventComment,County,Location,TenantAddress,TenantCity,TenantState,TenantZipCode,...,input_address,match_status,match_type,match_address,match_lon_lat,match_tiger_line_id,match_tiger_line_side,lon,lat,geometry
1382,1383.0,1/31/2023,Warrant of Restitution - Return of Service - E...,None,Prince George's,Hyattsville,9611 Converse Ct,Brandywine,MD,20613,...,"11215 Oak Leaf Drive APT 1809, Silver Spring, ...",Match,Exact,"11215 OAK LEAF DR, SILVER SPRING, MD, 20901","-76.995441638978,39.040597438831",608371134.0,R,-76.995441638978,39.040597438831,POINT (-76.99544 39.0406)
1383,1384.0,1/31/2023,Warrant of Restitution - Return of Service - E...,None,Prince George's,Hyattsville,920 Palmer Rd #06,Ft Washington,MD,20744,...,"3701 Twin Lake Court APT 323, Baltimore, MD, 2...",Match,Exact,"3701 TWIN LAKES CT, BALTIMORE, MD, 21244","-76.757914548052,39.360060650821",191467191.0,R,-76.757914548052,39.360060650821,POINT (-76.75791 39.36006)
1384,1385.0,1/31/2023,Warrant of Restitution - Return of Service - E...,None,Prince George's,Hyattsville,920 Palmer Rd #06,Ft Washington,MD,20744,...,"3701 Twin Lake Court APT 323, Baltimore, MD, 2...",Match,Exact,"3701 TWIN LAKES CT, BALTIMORE, MD, 21244","-76.757914548052,39.360060650821",191467191.0,R,-76.757914548052,39.360060650821,POINT (-76.75791 39.36006)
1386,1387.0,1/31/2023,Warrant of Restitution - Return of Service - E...,None,Prince George's,Hyattsville,918 Palmer Rd #06,Ft Washington,MD,20744,...,"11215 Oak Leaf Drive APT 1803, Silver Spring, ...",Match,Exact,"11215 OAK LEAF DR, SILVER SPRING, MD, 20901","-76.995441638978,39.040597438831",608371134.0,R,-76.995441638978,39.040597438831,POINT (-76.99544 39.0406)
1387,1388.0,1/31/2023,Warrant of Restitution - Return of Service - E...,None,Prince George's,Hyattsville,918 Palmer Rd #06,Ft Washington,MD,20744,...,"11215 Oak Leaf Drive APT 1803, Silver Spring, ...",Match,Exact,"11215 OAK LEAF DR, SILVER SPRING, MD, 20901","-76.995441638978,39.040597438831",608371134.0,R,-76.995441638978,39.040597438831,POINT (-76.99544 39.0406)


In [21]:
#2b: clean permit data
#only include 2015 - 2024 for permits
def tidy_permits(df):

 #conditions for existing & proposed uses
    existing_use_conditions = ['Vacant'], 
    proposed_use_conditions = ['Apartments']
    
    #apply filters
    filtered_df = df[
        df['EXISTING_USE'].isin(existing_use_conditions) &  
        df['PROPOSED_USE'].isin(proposed_use_conditions) 
    ]
    
    return filtered_df

df_permits_tidy = tidy_permits(epermits)
df_permits_tidy.head()

,PERMIT_NUMBER,CLASSIFICATION,REFERENCE_YEAR,APPLICATION_DATE,ISSUANCE_DATE,REVISION_ISSUED,WORK_DESCRIPTION,EXISTING_USE,PROPOSED_USE,USE_TYPE_NAME,...,STREET_TYPE_SUFFIX,STREET_TYPE_FULL,UNIT_NUMBER,CITY_1,CORRECTED_CITY,ZIP_CODE_5,CASE_STATUS_NAME,CASE_MODE_NAME,CASE_NAME,FEES_PAID


In [16]:
#3: join eviction, census, & permit data

#3a: joing census data
#combine geojsosn
combined_gdf = pd.concat([poverty, income, population], ignore_index=True)
#remove error columns
combined_gdf = combined_gdf.loc[:, ~combined_gdf.columns.str.contains('Error', case=False)]
combined_gdf.columns.tolist()

['geoid',
 'name',
 'B17001001',
 'B17001002',
 'B17001003',
 'B17001004',
 'B17001005',
 'B17001006',
 'B17001007',
 'B17001008',
 'B17001009',
 'B17001010',
 'B17001011',
 'B17001012',
 'B17001013',
 'B17001014',
 'B17001015',
 'B17001016',
 'B17001017',
 'B17001018',
 'B17001019',
 'B17001020',
 'B17001021',
 'B17001022',
 'B17001023',
 'B17001024',
 'B17001025',
 'B17001026',
 'B17001027',
 'B17001028',
 'B17001029',
 'B17001030',
 'B17001031',
 'B17001032',
 'B17001033',
 'B17001034',
 'B17001035',
 'B17001036',
 'B17001037',
 'B17001038',
 'B17001039',
 'B17001040',
 'B17001041',
 'B17001042',
 'B17001043',
 'B17001044',
 'B17001045',
 'B17001046',
 'B17001047',
 'B17001048',
 'B17001049',
 'B17001050',
 'B17001051',
 'B17001052',
 'B17001053',
 'B17001054',
 'B17001055',
 'B17001056',
 'B17001057',
 'B17001058',
 'B17001059',
 'geometry',
 'B19001001',
 'B19001002',
 'B19001003',
 'B19001004',
 'B19001005',
 'B19001006',
 'B19001007',
 'B19001008',
 'B19001009',
 'B19001010',
 '

In [20]:
#3b: join with geojsons
#4: spatial join eviction geodataframe to cleaned disability geojson
#confirm crs as UTM18N
utm18n = 26918
combined_gdf = combined_gdf.to_crs(epsg=utm18n)
eviction_tidy = eviction_tidy.to_crs(epsg=utm18n)
assert combined_gdf.crs == eviction_tidy.crs

#spatially join evictions with tracts
tracts_with_evictions_gdf = gpd.sjoin(combined_gdf, eviction_tidy, how='left')
tracts_with_evictions_gdf.head()

# Count unique evictions in each tract
eviction_counts = tracts_with_evictions_gdf.groupby('geoid').index_right.nunique()
eviction_counts

# Join counts onto tracts
tracts_with_counts_gdf = combined_gdf.merge(eviction_counts, left_on='geoid', right_index=True)

In [ ]:
#4: map

In [ ]:
#5: spatial analysis